# Prácticas

_Máster Big Data Analytics 2015 - 2020_  
_Luis Belloch bigdata@luisbelloch.es_  
_Universidad Politécnica de Valencia_  

El objetivo de la práctica es aprender a trasladar a Spark pequeños prototipos de algoritmos de cálculo que previamente hemos visto en clase mediante Pandas.

**IMPORTANTE**: 

1. La implementación de este documento sólo se incluye como referencia para poder entender los cálculos. Para los ejercicios únicamente puede utilizarse `PySpark` y la librería estándar de Python. Opcionalmente puedes hacer uso tambien de `Spark SQL`, pero no se permite utilizar `pandas` a no ser que el ejercicio lo indique. En algunos ejercicios se utiliza `stock_prices` para cargar los datos. Es sólo un ejemplo, debes utilizar Spark para la carga de datos.
2. Es posible obtener resultados ligeramente diferentes a los obtenidos mediante Pandas debido a errores de precisión.
3. Para el desarrollo del ejercicio puedes usar notebooks de python, pero debes entregar los ejercicios **en un único archivo .py** donde cada función corresponde a un ejercicio:

```
# ejercicios.py

def ejercicio1():
    ...
    
def ejercicio2():
    ...
```

## Ejercicio 1

**1 punto**

1. Mediante Pandas únicamente, y haciendo uso de `stock_prices` obten los datos de prueba para los simbolos `GOOG`, `AAPL` y `MSFT`.
2. Guarda los objetos de Pandas en formato CSV separado por comas.
3. Escribe un pequeño programa en Spark que lea los datos en CSV, separe por comas y elimine la primera de las filas en el caso de que el CSV contenga los nombres de las columnas.
4. Mediante Spark, filtra los datos de los años 2015 y 2016.

Recuerda que los datos de ejemplo están en la carpeta `data`, tanto en formato `MessagePack` como `CSV`.

## Ejercicio 2

**2 puntos**

Haciendo uso únicamente de Spark / Spark SQL:

1. Carga los datos para `MSFT` desde archivo descargado en el ejercicio 1.
2. Obtén el retorno entre los días 15 y 16 de junio del 2016. Recuerda que el retorno entre $t-1$ y $t$ lo definimos como $R_t = \frac{P_t - P_{t-1}}{P_{t-1}}$

In [1]:
%matplotlib inline
import pandas as pd
from sample_data import stock_prices
from datetime import datetime, timedelta
msft = stock_prices("msft")
msft.Close["2016-06-15":"2016-06-16"].pct_change()[-1:][0]

0.014087341517408092

## Ejercicio 3

**2 puntos**

Haciendo uso únicamente de Spark / Spark SQL, extrae una media del precio de cierre por cada cuatrimestre para 2015 y 2016.

In [2]:
msft.Close.resample("Q").mean()

Date
2010-03-31    29.277672
2010-06-30    27.961838
2010-09-30    24.759765
2010-12-31    26.344394
2011-03-31    27.011461
2011-06-30    25.078769
2011-09-30    26.069924
2011-12-31    26.055923
2012-03-31    30.469769
2012-06-30    30.444923
2012-09-30    30.300185
2012-12-31    27.941691
2013-03-31    27.682948
2013-06-30    32.784046
2013-09-30    32.922045
2013-12-31    36.352530
2014-03-31    37.516328
2014-06-30    40.459523
2014-09-30    44.576417
2014-12-31    46.968864
2015-03-31    43.627812
2015-06-30    45.565231
2015-09-30    44.862045
2015-12-31    52.684621
2016-03-31    52.444692
2016-06-30    51.970154
2016-09-30    56.385606
2016-12-31    60.198462
2017-03-31    64.043231
2017-06-30    68.611538
2017-09-30    72.984615
2017-12-31    82.100462
2018-03-31    91.336308
2018-06-30    96.589298
Freq: Q-DEC, Name: Close, dtype: float64

## Ejercicio 4

**2 puntos**

Haciendo uso únicamente de Spark / Spark SQL, computa el valor de un portfolio a final de mes. Recuerda que el retorno del porfolio para un periodo se define como:

$R_{p,t} = \sum\limits_{i=1}^n x_i R_{i,t}$

donde el porcentaje de participación en el portfolio es $x_n : x_1 + ... + x_n = 1$ y $R_{i,t}$ es cada retorno mensual de cada activo del portfolio.

Conociendo esto es posible extraer el valor del portfolio al final del mes $t$, donde $t$ es

$V_t = V_{t-1}(1 + R_{p,t})$


El portfolio de inicio tiene 10 acciones de `MSFT` y 10 acciones de `AAPL`, ambas compradas el 30 de marzo.

In [3]:
msft = stock_prices('msft')
aapl = stock_prices('aapl')
[msft.Close['2016-03-30'], aapl.Close['2016-03-30']]

[55.05, 109.56]

Los precios al final de més, donde computaremos el valor del portfolio, son:

In [4]:
[msft.Close['2016-04-29'], aapl.Close['2016-04-29']]

[49.87, 93.74]

Por lo que podemos calcular `initial_portfolio_value` de forma simple:

In [5]:
msft_shares = 10
aapl_shares = 10
initial_portfolio_value = (msft_shares * msft.Close['2016-03-30']) + (aapl_shares * aapl.Close['2016-03-30'])
initial_portfolio_value

1646.1

También necesitamos computar el peso relativo de cada inversión en el total. Observa que sumando ambos pesos nos da un 100% (puede resultarte interesante realizar esta comprobación en Spark):

In [6]:
x_msft = (msft_shares * msft.Close['2016-03-30']) / initial_portfolio_value
x_aapl = (aapl_shares * aapl.Close['2016-03-30']) / initial_portfolio_value
[x_msft, x_aapl, x_msft+x_aapl]

[0.3344268270457445, 0.6655731729542554, 1.0]

Desde aqui solo nos queda calcular el retorno de cada uno de los activos y el del propio portfolio:

In [7]:
ret_msft = msft.Close['2016-04-29'] / msft.Close['2016-03-30'] - 1
ret_aapl = aapl.Close['2016-04-29'] / aapl.Close['2016-03-30'] - 1
[ret_msft, ret_aapl]

[-0.09409627611262483, -0.14439576487769268]

In [8]:
portfolio_returns = (x_msft*ret_msft) + (x_aapl*ret_aapl)
portfolio_returns

-0.12757426644796796

Por lo tanto, el valor del portfolio al final del mes $t$ se puede obtener mediante:

In [9]:
vt = initial_portfolio_value * (1 + portfolio_returns)
vt

1436.1

## Ejercicio 5

**3 puntos**

Haciendo uso de Spark, computa la pérdida máxima sobre una cartera con un valor X y una probabilidad del 5%, para una fecha determinada.

El _valor en riesgo_ de una cartera es una medida que nos permite cuantificar las pérdidas en una inversión haciendo uso de herramientas estadísticas básicas. Para simplificar el ejercicio asumiremos que nuestra cartera esta compuesta de 1000 acciones de `AAPL` compradas a fecha de cierre del día 2 de febrero de 2015, por un valor de 106271.212€.

En este ejercicio calcularemos cúanto perderíamos el día siguiente con una probabilidad del 95%. Estadísticamente VaR se puede definir como:

${VaR} = P - P \cdot (\alpha(1-c) + 1)$, siendo $c$ el intervalo de confianza

Lo primero que haremos será obtener los precios históricos de cierre para `AAPL` desde Google Finance.

In [10]:
aapl = stock_prices('aapl')
prices = aapl.Close
prices["2015-01-02"]

109.33

A partir de aqui podemos calcular el valor del portfolio en esa fecha. Como el porfolio está compuesto de un solo activo, el cálculo es muy sencillo:

In [11]:
shares = 1000
portfolio_value = shares * prices.loc['2015-1-02']
portfolio_value

109330.0

Lo siguiente que realizaremos será computar los retornos diarios para toda la serie de valores historicos. Recuerda que estamos utilizando la columna `Adj Close`.

In [12]:
import numpy as np

def daily_returns(closes):
    return np.log(closes / closes.shift(1))

returns = daily_returns(prices)
returns[-3:]

Date
2018-06-14    0.000524
2018-06-15   -0.010326
2018-06-18   -0.000530
Name: Close, dtype: float64

Desde aqui únicamente nos queda extraer el z-score a aplicar y sustituir valores en la fórmula de VaR descrita arriba:

In [13]:
from scipy.stats import norm
z_score = norm.ppf(0.95) # loc:0 (µ), scale: 1 (σ)
alpha = z_score * returns.std()
value_at_risk = portfolio_value * alpha
value_at_risk

2814.289349489219

Para simplificar el ejercicio puedes utilizar una constante para `z-score` en lugar de calcularlo dentro del proceso. Igualmente es importante que no hagas uso de numpy o scipy para realizar los cálculos en Spark.